# Part 1: Horse Racing Prediction

 ## Data Preprocessing

In [ ]:
!conda install -c rapidsai -c nvidia -c numba -c conda-forge \
   cudf=21.08 python=3.9 cudatoolkit=11.0

In [ ]:
import cudf 
import pandas as pd
import numpy as np

# rcParams
# change image to white background
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'white'

%load_ext autotime

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read the data
rrh = pd.read_csv('./data/race-result-horse.csv')
rrr = pd.read_csv('./data/race-result-race.csv')
print(rrh.shape)
print(rrr.shape)

In [ ]:
# View the data
rrh.head()

In [ ]:
# View the data
rrr.head()

In [ ]:
# Explore the data
rrh.info()

In [ ]:
# Explore the data
rrr.info()

## Data Cleaning

In [ ]:
# Look at the unique values in the column 'finishing_position'
rrh['finishing_position'].unique()

In [ ]:
# Drop rows in column where value is not a number
df_horse = rrh[rrh['finishing_position'].isin([str(i) for i in range(1, 15)])].reset_index(drop=True)

In [ ]:
# View the data
df_horse.head()

In [ ]:
# Check the data types after removing special characters
df_horse.finishing_position.unique()

In [ ]:
df_horse

In [ ]:
# #  drop the rows where the “finish_position” is not a number (e.g. WV-A, WV). There may be accidents.
# rrh = rrh[rrh.finishing_position.isin(['1','2','3','4','5','6','7','8','9','10','11','12','13','14'])].reset_index()
# rrh.head()

In [ ]:
horse_id = df_horse['horse_id'].unique()
numHorse = len(horse_id)
horse_index = range(numHorse)

jockey = df_horse['jockey'].unique()
numJockey = len(jockey)
jockey_index = range(numJockey)

trainer = df_horse['trainer'].unique()
numTrainer = len(trainer)
trainer_index = range(numTrainer)

print(' Number of Horses:', numHorse, '\n', 'Number of Jockeys:', numJockey,'\n', 'Number of Trainers:', numTrainer)

In [ ]:
# Takes a while to run

# Add a column named recent_6_runs to the dataframe, which records the recent ranks of the horse in each entry. 
# The ranks are separated by “/”, and a record is like 1/2/6/3/4/7.

# Add a column named recent_ave_rank for each entry to the dataframe, 
# which records the average rank of the recent 6 runs of a horse

df_horse['recent_6_runs'] = '0'
df_horse['recent_ave_rank'] = '7'

for i in range(len(df_horse['finishing_position'])):
    
    # temp will just give a list of all the finishing positions of that particular horse
    temp = df_horse[ : (i + 1)][df_horse.horse_id == df_horse.horse_id[i]][['finishing_position']]

    # we convert temp into a list, in reverse order, with the last race in front
    temp = temp['finishing_position'].values.tolist()[::-1]
  
    # we then take the first 6 elements of the list, and join them with a '/'
    df_horse['recent_6_runs'][i] = '/'.join(temp[:6])
    
    # if list is not empty, then we convert string to float and take the average to get the recent_ave_rank
    if len(temp) != 0:
        temp_int = map(int,temp)
        temp_ave = np.mean(list(temp_int))
        df_horse['recent_ave_rank'][i] = temp_ave

In [ ]:
# Check the data
df_horse.tail()

In [ ]:
# Add column of Distance

# The distance could be 1000, 1200, 1400, 1600, 1800, 2000, 2400, etc. 
# Some horses are good at short-distance races, while some are good at long-distance races.

df_horse['race_distance'] = '0'

# Iterate through the dataframe
for i in range(len(rrr['race_id'])):
    distance = rrr['race_distance'][i]
    df_horse['race_distance'][df_horse.race_id == rrr.race_id[i]] = distance

In [ ]:
# Add HorseWin, HorseRankTop3, HorseRankTop50Percent for use in Part3

# Sets a boolean mask (either 1 or 0) for each new column
df_horse['HorseWin'] = (df_horse.finishing_position == '1') + 0
df_horse['HorseRankTop3'] = (df_horse.finishing_position == '1') + (df_horse.finishing_position == '2') + (df_horse.finishing_position == '3') + 0

# Create a list of all the top1_indexes where the horse finished in the top
top1_index = df_horse.index[df_horse['finishing_position'] == '1'].tolist()

# Create another column where the horse finished in the top 50%
df_horse['HorseRankTop50Percent'] = '0'
for i in range(len(top1_index)-1):
    df_horse['HorseRankTop50Percent'][top1_index[i]:(top1_index[i] + int(round(0.5 * (top1_index[i + 1] - top1_index[i]))))] = '1'

# Created specifically for the last index because we cannot loop    
df_horse['HorseRankTop50Percent'][top1_index[len(top1_index) - 1]:(top1_index[len(top1_index) - 1] + 6)] = '1'


In [ ]:
df_horse.head()

In [ ]:
# Save the data
df_horse.to_csv('./data/df_horse.csv', index=False)

## Data Preparation

In [ ]:
# Divide data into train set, test set and unseen backtest set.
# Training set contains all the races with race_id no more than “2016-327” (first 80%), 
# Test set contains the rest of races (last 20%)

train_last_index = max(df_horse.index[df_horse['race_id'] == '2016-137'].tolist())
test_last_index = max(df_horse.index[df_horse['race_id'] == '2016-563'].tolist())

print(train_last_index)
print(test_last_index)


df_train = df_horse[:(train_last_index + 1)]
df_test = df_horse[(train_last_index + 1):(test_last_index + 1)]
df_unseen = df_horse[(test_last_index + 1):]


In [ ]:
# Calculate jockey_ave_rank, trainer_ave_rank ONLY for training and test data
# jockey_ave_rank: records the average rank of the jockey
# trainer_ave_rank: records the average rank of the trainer

def cal_ave_rank(df):
    df['jockey_ave_rank'] = '7'

    for i in range(len(jockey)):
        temp = df[df.jockey == jockey[i]][['finishing_position']]
        temp = temp['finishing_position'].values.tolist()
        if len(temp) != 0:
            temp_int = map(int, temp)
            temp_ave = np.mean(list(temp_int))
            df['jockey_ave_rank'][df.jockey == jockey[i]] = temp_ave


    df['trainer_ave_rank'] = '7'

    for i in range(len(trainer)):
        temp = df[df.trainer == trainer[i]][['finishing_position']]
        temp = temp['finishing_position'].values.tolist()
        if len(temp) != 0:
            temp_int = map(int,temp)
            temp_ave = np.mean(list(temp_int))
            df['trainer_ave_rank'][df.trainer == trainer[i]] = temp_ave

    return df


In [ ]:
# Save the train set
df_train = cal_ave_rank(df_train)
df_train.to_csv('./data/df_train.csv', index=False)

# Save the test set
df_test = cal_ave_rank(df_test)
df_test.to_csv('./data/df_test.csv', index=False)

In [ ]:
# Port over the jockey_ave_rank and trainer_ave_rank to the unseen set

df_unseen['jockey_ave_rank'] = '7'
df_unseen_jockey = df_unseen['jockey'].unique()
for i in range(len(df_unseen_jockey)):
    if df_unseen_jockey[i] in df_train['jockey'].unique():
        df_unseen['jockey_ave_rank'][df_unseen.jockey == df_unseen_jockey[i]]\
            = df_train[df_train.jockey == df_unseen_jockey[i]]['jockey_ave_rank'].tolist()[0]

df_unseen['trainer_ave_rank'] = '7'
df_unseen_trainer=df_unseen['trainer'].unique()
for i in range(len(df_unseen_trainer)):
    if df_unseen_trainer[i] in df_train['trainer'].unique():
        df_unseen['trainer_ave_rank'][df_unseen.trainer == df_unseen_trainer[i]]\
            = df_train[df_train.trainer == df_unseen_trainer[i]]['trainer_ave_rank'].tolist()[0]

In [ ]:
df_unseen.to_csv('./data/df_unseen.csv', index=False)